# Competition — Marketing Campaign

### Challenge Overview

You are running the marketing campaign for a brand new pocket device. Initially you can sign contracts with a few people to advertize your gadget among their neigbours. The more "famous" person you are picking the greater price appears in the contract. Contract cost can be calculated as 300\\$ * NN(i), where NN(i) is the number of neigbours of the person i. If at least 18\% a person's neighbors have already been affected, then he/she will also be affected tomorrow. You earn 50\\$ per each affected person. Every day you have to choose whether to sign new contracts or wait. You need time to discuss terms of a contract, so you cannot sign more than 10 contracts on the same day. Your task is to maximize profit of your campaign with an initial budget of 10,000\\$. Your campaign is considered completed after 60 days.

Again, all parameters of the task:
* Budget: 10,000\\$
* Contract cost: 300$ * NN(i)
* Income per person: 50\\$
* Exposure threshold: 18%
* Contracts limit: 10 per day
* Time limit: 60 days

A model of society is based on undirected SNAP Facebook network `edge_list.txt`

### Evaluation Criteria

On the leaderboard you will see three scores:
* Profit — calculated as "total amount of money at the end - initial budget". This is the main score.
* Accepted — the total number of accepted people at the end.
* Days — the total duration of your campaign.

**Baselines**

Baselines are calculated as follows:
1. Simultaneously sign 10 random contracts on the first day. Decrease the number of contracts if you do not have enough money
1. Wait the end of the campaign and calculate your profit.
1. Repeat 10000 times

Baseline for grade 4: beat an average positive profit

Baseline for grade 6: beat an average + standard deviation positive profit

Baseline for grade 8: beat a maximum positive profit

### Submission Guidelines

Submit a txt with days and contracts which will be signed. That looks like a python dict object, the key is a day, the value is a list of counterparties. Numbering of days from 0. 

**Example 1**
```
{
    # On the first day, I will sign contracts with 47 and 138
    0: [47, 138],
    # On the third day, I will sign 2 more contracts
    2: [3789, 262]
}
```

Result of this campaign: 

* Profit: -2350
* Days: 10
* Affected: 29

**Example 2**
```
{
    # I do not want to do anything
}
```

Result of this campaign: 
* Profit: 0
* Days: 1
* Affected: 0

**Exceptions**

You may encounter two types of exceptions after submission:
1. You cannot sign more than 10 contracts at the same day.
1. Not enough money for the contract with 2919. The contract cost is 300\\$, you have only 100\\$.

In this case, the evaluation will be interrupted and your scores will not be calculated. You can find log files with detailed information on the My Submissions page, stdout file. To sign a contract with an already affected person is not a problem, you will just waste your money.

In [1]:
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm

In [2]:
def contract_cost(G, node):
    num_neighbours = len(list(G.neighbors(node)))
    return 300 * num_neighbours


def make_affected(G, node):
    # True if we should make node affected
    neighbours = list(G.neighbors(node))
    num_neighbours = len(neighbours)
    num_affected = len([n for n in neighbours if G.nodes[n]["affected"] == 1])
    return num_affected / num_neighbours >= 0.18


def get_num_affected(G):
    affected = [n for n in G.nodes if G.nodes[n]["affected"] == 1]
    num_affected = len(affected)
    return num_affected


def sign_contracts(G, available_budget, affected_nodes=None):
    nodes = list(G.nodes)
    
    asd = {node: len(list(G.neighbors(node))) for node in G.nodes}
    nodes = list(dict(sorted(asd.items(), key=lambda item: item[1], reverse=True)).keys())
    
    chosen_nodes = []
    cost_of_contracts = 0
    
    while available_budget >= cost_of_contracts and len(chosen_nodes) <= 10 and nodes:
        node = np.random.choice(nodes)
        cost = contract_cost(G, node)
        if available_budget >= (cost_of_contracts + cost) and node not in affected_nodes:
            chosen_nodes.append(node)
            cost_of_contracts += cost
        nodes.remove(node)
    return chosen_nodes, cost_of_contracts

In [ ]:
np.random.seed(120)

G = nx.read_edgelist("edge_list.txt")
nx.set_node_attributes(G, 0, name="affected")

G = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])

budget = 10000
contract_limit = 10
time_limit = 60
customer_income = 50

nodes = list(G.nodes)
affected_nodes = []

trace = dict()

for t in tqdm(range(time_limit)):
    cost_of_contracts = 0
    for node in nodes:
        if make_affected(G, node):
            G.nodes[node]["affected"] = 1
    if t < 50:
        signed_contracts, cost_of_contracts = sign_contracts(G, budget, affected_nodes)

        for node in signed_contracts:
            G.nodes[node]["affected"] = 1

        trace[t] = signed_contracts
    
    num_new_affected = len([node for node in G.nodes if G.nodes[node]["affected"] == 1]) - len(affected_nodes)
    affected_nodes = [node for node in G.nodes if G.nodes[node]["affected"] == 1]
    
    
    budget = budget + num_new_affected * customer_income - cost_of_contracts
    print(budget)

250
300
100
150
350
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [ ]:
trace

In [184]:
trace = {k: [int(x) for x in v] for k, v in trace.items() if v}
trace

{0: [3870, 1797, 164, 228],
 2: [1233],
 4: [3125],
 6: [316],
 8: [377],
 9: [35],
 10: [2382],
 11: [1394],
 12: [279],
 13: [52],
 14: [2798],
 15: [443],
 16: [71, 154],
 17: [47],
 18: [1693, 3801, 1546],
 19: [182, 160],
 20: [3688, 3626],
 21: [81, 1065],
 22: [3805, 3807, 1022, 468],
 23: [4036, 3375, 255],
 24: [448, 1326, 3844, 2168],
 25: [234, 124, 3928],
 26: [4013, 1264, 2595, 3959],
 27: [3781, 371, 240],
 28: [3661, 121],
 29: [3504, 4011, 1586, 3457],
 30: [131, 3770, 3977, 3875, 216],
 31: [937, 1838, 3091, 307],
 32: [3519, 3133, 3922],
 33: [1430, 2831, 70, 2975, 2740],
 34: [132, 1353, 4032, 2963, 32, 2178, 411],
 35: [598, 293, 1276, 189, 379],
 36: [3014, 3952, 3942],
 37: [536, 2568, 2885, 2736],
 38: [2610, 3572],
 39: [2385, 4017, 76],
 40: [592, 3806, 866, 153, 3444, 27, 3589],
 41: [2502, 3653, 1239, 1892, 2380, 253],
 42: [62, 3960, 593, 89, 533, 1713, 267],
 43: [2733, 3767, 2523, 100, 3445, 3936, 671],
 44: [194, 3668, 2923, 2599, 485, 247, 3808],
 45: [2

In [185]:
with open("sub.txt", "w") as f:
    f.write(str(trace))

In [173]:
get_num_affected(G)

816

In [100]:
len(list(G.neighbors('497'))) * 300

24600

In [92]:
signed_contracts

['237', '497']

In [90]:
cost_of_contracts

26400

In [34]:
nx.set_node_attributes(G, 0, name="affected")

In [47]:
G.nodes["1"]["affected"]

0

In [53]:
make_affected(G, "1")

True